# Hops - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 22, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [6]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [7]:
# connect to SQL Server.
SERVER = '34.206.73.189' 
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [8]:
lkupclientid = 9 # Hops

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrMILB].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,9,410730463,2335348,2017,Mini Plans / Flex Plans,150.0,2017-08-06,0.800000,93,TRUE,2555,193,21.900000,5,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
1,9,410730544,3143685,2017,Mini Plans / Flex Plans,136.0,2017-08-18,1.000000,53,TRUE,2190,102,35.790000,4,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
2,9,410731033,1879414,2017,Mini Plans / Flex Plans,120.0,2017-08-30,1.333333,102,TRUE,2920,173,16.260000,4,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
3,9,410734282,411882,2017,Full Season,1342.0,2017-08-30,0.500000,214,TRUE,3285,285,5.370000,19,0,3,5,2,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1
4,9,457208280,1879982,2017,Full Season,1184.0,2017-08-30,0.684211,257,TRUE,2920,328,135.933254,29,0,5,2,2,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,75,None,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3077 entries, 0 to 3076
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            3077 non-null   int64  
 1   dimCustomerMasterId     3077 non-null   int64  
 2   customerNumber          3077 non-null   object 
 3   year                    3077 non-null   int64  
 4   productGrouping         3077 non-null   object 
 5   totalSpent              3077 non-null   float64
 6   recentDate              3077 non-null   object 
 7   attendancePercent       3077 non-null   float64
 8   renewedBeforeDays       3077 non-null   int64  
 9   isBuyer                 3077 non-null   object 
 10  source_tenure           3077 non-null   int64  
 11  tenure                  3077 non-null   int64  
 12  distToVenue             3077 non-null   float64
 13  totalGames              3077 non-null   int64  
 14  recency                 3077 non-null   

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [10]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(3077, 20)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [11]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3077 entries, 0 to 3076
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dimCustomerMasterId     3077 non-null   int64  
 1   totalSpent              3077 non-null   float64
 2   attendancePercent       3077 non-null   float64
 3   renewedBeforeDays       3077 non-null   int64  
 4   source_tenure           3077 non-null   int64  
 5   tenure                  3077 non-null   int64  
 6   totalGames              3077 non-null   int64  
 7   is_Lockdown             3077 non-null   int64  
 8   NumberofGamesPerSeason  3077 non-null   int64  
 9   isNextYear_Buyer        3077 non-null   int64  
dtypes: float64(2), int64(8)
memory usage: 240.5 KB


### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [12]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,dimCustomerMasterId,totalSpent,attendancePercent,renewedBeforeDays,source_tenure,tenure,totalGames,is_Lockdown,NumberofGamesPerSeason,isNextYear_Buyer
dimCustomerMasterId,1.00,0.16,-0.02,-0.01,0.05,0.05,0.02,0.01,-0.00,0.10
totalSpent,0.16,1.00,-0.24,0.06,0.09,0.16,0.43,0.07,0.09,0.07
attendancePercent,-0.02,-0.24,1.00,0.15,0.09,-0.25,0.05,-0.27,-0.28,0.10
renewedBeforeDays,-0.01,0.06,0.15,1.00,0.32,-0.13,0.34,-0.49,-0.21,0.06
source_tenure,0.05,0.09,0.09,0.32,1.00,0.30,0.37,-0.17,-0.15,0.19
tenure,0.05,0.16,-0.25,-0.13,0.30,1.00,0.24,0.64,0.45,0.10
totalGames,0.02,0.43,0.05,0.34,0.37,0.24,1.00,-0.01,-0.02,0.26
is_Lockdown,0.01,0.07,-0.27,-0.49,-0.17,0.64,-0.01,1.00,0.35,0.17
NumberofGamesPerSeason,-0.00,0.09,-0.28,-0.21,-0.15,0.45,-0.02,0.35,1.00,-0.44
isNextYear_Buyer,0.10,0.07,0.10,0.06,0.19,0.10,0.26,0.17,-0.44,1.00


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [13]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (2462, 10)
Unseen Data For Predictions: (615, 10)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [17]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "totalSpent",
        "attendancePercent",
        "renewedBeforeDays",
        "source_tenure",
        "tenure",
        "totalGames",
        "is_Lockdown",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,3541
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(2462, 10)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


(None,
 False,
 'lightgbm',
 {'parameter': 'Hyperparameters',
  'auc': 'AUC',
  'confusion_matrix': 'Confusion Matrix',
  'threshold': 'Threshold',
  'pr': 'Precision Recall',
  'error': 'Prediction Error',
  'class_report': 'Class Report',
  'rfe': 'Feature Selection',
  'learning': 'Learning Curve',
  'manifold': 'Manifold Learning',
  'calibration': 'Calibration Curve',
  'vc': 'Validation Curve',
  'dimension': 'Dimensions',
  'feature': 'Feature Importance',
  'feature_all': 'Feature Importance (All)',
  'boundary': 'Decision Boundary',
  'lift': 'Lift Chart',
  'gain': 'Gain Chart',
  'tree': 'Decision Tree',
  'ks': 'KS Statistic Plot'},
 Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                              

In [18]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7963,0.8844,0.7279,0.6971,0.7112,0.5542,0.5555,0.2750
lr,Logistic Regression,0.6546,0.5204,0.0000,0.0000,0.0000,0.0000,0.0000,0.3470


In [19]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8020,0.8846,0.7059,0.7164,0.7111,0.5605,0.5606
1,0.7716,0.8691,0.6618,0.6716,0.6667,0.4929,0.4930
2,0.8376,0.9067,0.8382,0.7308,0.7808,0.6527,0.6566
3,0.8122,0.9005,0.7941,0.7013,0.7448,0.5971,0.6000
4,0.7614,0.8619,0.6618,0.6522,0.6569,0.4741,0.4741
5,0.7868,0.8952,0.7206,0.6806,0.7000,0.5349,0.5354
6,0.7766,0.8651,0.7353,0.6579,0.6944,0.5193,0.5213
7,0.7919,0.8796,0.7647,0.6753,0.7172,0.5536,0.5563
8,0.8223,0.8939,0.7059,0.7619,0.7328,0.6000,0.6010
9,0.8010,0.8875,0.6912,0.7231,0.7068,0.5563,0.5566


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?